In [4]:
from deepface import DeepFace
import pygame
import time
# Initialize mixer
pygame.mixer.init()
pygame.mixer.music.set_volume(0.5)  # Initial volume

# Music by emotion
emotion_music = {
    'happy': 'happy.mp3',
    'sad': 'sad.mp3',
    'angry': 'angry.mp3',
    'surprise': 'surprise.mp3',
    'neutral': 'calm.mp3'
}

# Play music by emotion
def play_music(emotion):
    if emotion in emotion_music:
        pygame.mixer.music.load(emotion_music[emotion])
        pygame.mixer.music.play()

# Volume control
def increase_volume():
    vol = pygame.mixer.music.get_volume()
    pygame.mixer.music.set_volume(min(1.0, vol + 0.1))
    print(f"Volume up: {pygame.mixer.music.get_volume():.2f}")

def decrease_volume():
    vol = pygame.mixer.music.get_volume()
    pygame.mixer.music.set_volume(max(0.0, vol - 0.1))
    print(f"Volume down: {pygame.mixer.music.get_volume():.2f}")

# Init
cap = cv2.VideoCapture(0)
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)
drawing = mp.solutions.drawing_utils

last_emotion = None
last_checked = time.time()
cooldown = 5
song_changed_on_left = False

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    h, w, _ = frame.shape

    if results.multi_face_landmarks:
        for face in results.multi_face_landmarks:
            # Extract landmarks for eyes
            left_eye = face.landmark[386]
            right_eye = face.landmark[159]

            # Convert to pixel coordinates
            lx, ly = int(left_eye.x * w), int(left_eye.y * h)
            rx, ry = int(right_eye.x * w), int(right_eye.y * h)

            cv2.circle(frame, (lx, ly), 4, (0, 255, 0), -1)
            cv2.circle(frame, (rx, ry), 4, (255, 0, 0), -1)

            # Basic eye raise detection
            # If eye is high (lower y-value), consider it "raised"
            if ly < h * 0.35:
                increase_volume()
                time.sleep(0.5)
            elif ry < h * 0.35:
                decrease_volume()
                time.sleep(0.5)

            # Get face ROI
            x_min = min(int(l.x * w) for l in face.landmark)
            y_min = min(int(l.y * h) for l in face.landmark)
            x_max = max(int(l.x * w) for l in face.landmark)
            y_max = max(int(l.y * h) for l in face.landmark)
            face_roi = frame[y_min:y_max, x_min:x_max]

            # Face center to detect turn
            face_center_x = (x_min + x_max) // 2
            frame_center_x = w // 2
            face_turn = "center"
            if face_center_x < frame_center_x - 50:
                face_turn = "left"
            elif face_center_x > frame_center_x + 50:
                face_turn = "right"

            # Emotion detection on cooldown
            if time.time() - last_checked >= cooldown:
                try:
                    result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
                    emotion = result[0]['dominant_emotion']
                    print(f"Emotion: {emotion}, Face turned: {face_turn}")

                    if emotion != last_emotion or (face_turn == "left" and not song_changed_on_left):
                        play_music(emotion)
                        last_emotion = emotion
                        song_changed_on_left = (face_turn == "left")

                    last_checked = time.time()
                except Exception as e:
                    print("Emotion detection error:", e)

    cv2.imshow("AI Music Controller", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
pygame.mixer.music.stop()


ValueError: You have tensorflow 2.19.0 and this requires tf-keras package. Please run `pip install tf-keras` or downgrade your tensorflow.